# Analysis of Dispatch vs. Demand
## 1. Introduction

This Notebook investigates how dispatch from models satisfies demand. We'll load various datasets, perform comparisons, and analyze results through visualizations.

## 2. Import Libraries

In [64]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

## 3. Load and Prepare Data

In [65]:
model_inputs_path = '../../model/inputs/'
model_outputs_path = '../../model/outputs/'
dema_path = '../../data/XM-API/variable_query/2022-12-01_2023-11-30/'

### 3.1 Timepoints
It is always advisable to load the timepoints from the model to simplify the transformations of many tables when merging.

In [66]:
# Load timepoints
timepoints = pd.read_csv(model_inputs_path+'timepoints.csv')
timepoints['timepoints'] = timepoints['timepoint_id']
timepoints = timepoints.drop(columns=['timeseries','timepoint_id'])

### 3.2.1 Dispatch by Tecnology from Switch
`dispatch_system` will be compared with the system registry on xm, while `dispatch` will be used to compare by technology.

In [67]:
dispatch_sw = pd.read_csv(model_outputs_path+'dispatch.csv')
dispatch_sw = dispatch_sw.groupby(['timestamp','gen_tech']).agg({
    'DispatchGen_MW' : 'sum'
}).reset_index()
dispatch_sw = pd.merge(dispatch_sw, timepoints, on='timestamp', how='inner')
dispatch_sw = dispatch_sw.sort_values(by='timepoints')

dispatch_sw.head(3)

,timestamp,gen_tech,DispatchGen_MW,timepoints
120,2023_Q1_labor_0h,Eolica,26.734497,1
121,2023_Q1_labor_0h,Hidro,6852.668141,1
124,2023_Q1_labor_0h,pv_solar,0.000000,1


### 3.2.1 Dispatch by Tecnology from XM

In [68]:
dispatch_xm = pd.read_csv(dema_path+'Melted_Gen_Res.csv')
dispatch_xm = dispatch_xm.drop(dispatch_xm.columns[0], axis=1)
dispatch_xm['DispatchGen_MW'] = dispatch_xm['GeneReal'] / 1000
# Add Timepoints
dispatch_xm = pd.merge(dispatch_xm, timepoints, on='timestamp', how='inner')
# Sort by timepoints
dispatch_xm = dispatch_xm.sort_values(by='timepoints')

dispatch_xm.head(3)

,timestamp,Values_Type,GeneReal,DispatchGen_MW,timepoints
120,2023_Q1_labor_0h,COGENERADOR,1.060746e+05,106.074569,1
123,2023_Q1_labor_0h,SOLAR,1.675325e-02,0.000017,1
124,2023_Q1_labor_0h,TERMICA,1.499315e+06,1499.314782,1


### 3.3.1 Dispatch by System from Switch

In [69]:
dispatch_sys = dispatch_sw.groupby(['timestamp','timepoints']).agg({
    'DispatchGen_MW' : 'sum'
    }).reset_index()

### 3.3.2 Demand by System from Switch

In [70]:
loads = pd.read_csv(model_inputs_path+'loads.csv')
loads = loads.groupby(['timepoints']).agg({
    'zone_demand_mw': 'sum'
}).reset_index()

print(loads.shape)
loads.head(3)

(384, 2)


,timepoints,zone_demand_mw
0,1,8433.925584
1,2,7961.776883
2,3,7689.767922


### 3.3.3 Dispatch and Demand by System from XM

In [71]:
XM = pd.read_csv(dema_path+'Melted_DemaGen_Sys.csv')
# Group by timestamp
XM = XM.groupby(['timestamp']).agg({
    'DemaReal_Sistema': 'mean',
    'DemaCome_Sistema': 'mean',
    'Gene_Sistema': 'mean',
    'GeneIdea_Sistema': 'mean'
}).reset_index()
# Adjust to MW
XM['DemaReal_Sistema'] = XM['DemaReal_Sistema'] / 1000
XM['DemaCome_Sistema'] = XM['DemaCome_Sistema'] / 1000
XM['Gene_Sistema'] = XM['Gene_Sistema'] / 1000
XM['GeneIdea_Sistema'] = XM['GeneIdea_Sistema'] / 1000

print(XM.shape)
XM.head(3)

(192, 5)


,timestamp,DemaReal_Sistema,DemaCome_Sistema,Gene_Sistema,GeneIdea_Sistema
0,2023_Q1_holidays_0h,7915.745849,8076.836295,8076.777236,8076.777236
1,2023_Q1_holidays_10h,7725.791177,7856.700819,7856.700819,7856.700819
2,2023_Q1_holidays_11h,8022.721860,8154.997578,8154.919696,8154.919696


## 3.4 System Analysis

### 3.4.1 Is Demand on Switch met by Switch Dispatch?

In [72]:
# Merge tables, result will be: timepoints, timestamp, zone_demand_mw, dispatch_wide 
demand_x_dispatch = pd.merge(loads, dispatch_sys, on='timepoints', how='inner')
# Sort by Timepoints
demand_x_dispatch = demand_x_dispatch.sort_values(by='timepoints')
# Rename columns for better undestanding
demand_x_dispatch.rename(columns={'DispatchGen_MW': 'Dispatch', 'zone_demand_mw': 'Demand'}, inplace=True)

fig = px.line(
    demand_x_dispatch,
    x='timestamp',
    y=['Dispatch', 'Demand'],
    labels={'value': 'Value (MW)', 'variable': 'Series'})
fig.update_layout(plot_bgcolor='white',xaxis=dict(gridcolor='lightgray'),yaxis=dict(gridcolor='lightgray'))
fig.show()

### 3.4.2 Are Demand and Dispatch on XM similar to Switch Dispatch?

In [73]:
# Merge tables
demand_x_dispatch = pd.merge(XM, dispatch_sys, on='timestamp', how='inner')
import plotly.express as px
# Sort by timepoints to sort timestamps as well
demand_x_dispatch = demand_x_dispatch.sort_values(by='timepoints')
fig = px.line(
    demand_x_dispatch,
    x='timestamp',
    y=['DemaReal_Sistema', 'DemaCome_Sistema', 'Gene_Sistema', 'GeneIdea_Sistema', 'DispatchGen_MW'],
    labels={'value': 'Values','variable': 'Series'},
    title='Demand/Dispatch XM vs Dispatch Switch'
)
fig.show()

### 3.4.3 Dispatch by Technology XM

In [74]:
gen_res = dispatch_xm.copy()
gen_res['Tech'] = gen_res['Values_Type']
# Change values on Tech to match Switch output
gen_res['Tech'] = gen_res['Tech'].replace({
        'EOLICA': 'Eolica', 'TERMICA': 'Thermal',
        'SOLAR': 'Solar','HIDRAULICA': 'Hidro',
        'COGENERADOR': 'Menores'})
# Replace prefix 'labor_' by 'L_' and 'holidays_' by 'H_'
gen_res['timestamp'] = gen_res['timestamp'].str.replace('labor_', 'L_')
gen_res['timestamp'] = gen_res['timestamp'].str.replace('holidays_', 'H_')

import plotly.express as px
fig = px.line(
    gen_res,
    x='timestamp', y='DispatchGen_MW',
    color='Tech', title='Generation by Technology (XM)',
    labels={'GeneReal': 'Reported Generation (MW)'}
)
fig.update_layout(plot_bgcolor='white',xaxis=dict(gridcolor='lightgray'),yaxis=dict(gridcolor='lightgray'))
fig.show()

### 3.4.4 Dispatch by Technology Switch

In [75]:
gen_disp = dispatch_sw.copy()
# Add Timepoints
gen_disp = pd.merge(gen_disp, timepoints, on='timestamp', how='inner')
# Sort by timepoints to sort timestamps as well
gen_disp = gen_disp.sort_values(by='timepoints_x')
gen_disp.rename(columns={'timepoints_x': 'timepoints'}, inplace=True)

# Change values on gen_tech to match Switch output
gen_disp['gen_tech'] = gen_disp['gen_tech'].replace({
    'Eolica': 'Eolica', 'Thermal': 'Thermal',
    'pv_solar': 'Solar', 'Hidro': 'Hidro',
    'Menores': 'Menores'})
# Replace prefix 'labor_' by 'L_' and 'holidays_' by 'H_'
gen_disp['timestamp'] = gen_disp['timestamp'].str.replace('labor_', 'L_')
gen_disp['timestamp'] = gen_disp['timestamp'].str.replace('holidays_', 'H_')

gen_disp['Tech'] = gen_disp['gen_tech']
fig = px.line(
    gen_disp, 
    x='timestamp', y='DispatchGen_MW', 
    color='Tech', title='Generation by Technology (Switch)'
)
fig.update_layout(plot_bgcolor='white',xaxis=dict(gridcolor='lightgray'),yaxis=dict(gridcolor='lightgray'))
fig.show()

### 3.4.5 Dispatch by Technology Switch x XM

In [76]:
# Take both Sets
gen_res = gen_res[['timestamp', 'Tech', 'DispatchGen_MW', ]]
gen_disp = gen_disp[['timestamp', 'Tech', 'DispatchGen_MW']].iloc[0:960]

# Define colors by Technology
tech_colors = {'Eolica': '#45CFFF',
               'Thermal': '#E84A59',
               'Solar': '#EFB52D',
               'Hidro': '#744FD0',
               'Menores': '#AEE83C'}

# Crete individual plots
fig1 = px.line(
    gen_res, x='timestamp', y='DispatchGen_MW', 
    color='Tech', title='Generation Reported (XM)',
    color_discrete_map=tech_colors)
fig2 = px.line(
    gen_disp, x='timestamp', y='DispatchGen_MW', 
    color='Tech', title='Generation Predicted (Switch)',
    color_discrete_map=tech_colors)

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Generation Reported (XM)", "Generation Predicted (Switch)"))
# Add figures to subplots
for trace in fig1['data']:
    fig.add_trace(trace, row=1, col=1)
for trace in fig2['data']:
    fig.add_trace(trace, row=1, col=2)

# Only show one legend
names = set()
for trace in fig['data']:
    if (trace.name in names): trace.showlegend = False
    else: names.add(trace.name)

# Add buttons to change y-range
fig.update_layout(
    title='Base Year (2023)',
    updatemenus=[
        go.layout.Updatemenu(
            buttons=list([
                dict(
                    args=[{"yaxis.autorange": True, "yaxis2.autorange": True}],
                    label="Auto",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.range": [0, 700], "yaxis2.range": [0, 7000]}],
                    label="700k Fixed",
                    method="relayout"
                )]),
            direction="down",
            x=1.2, # Adjust horizontal position
            xanchor="right",
            y=1.2  # Adjust vertical position
        )],
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    xaxis2=dict(showgrid=True, gridcolor='lightgray'),
    yaxis2=dict(showgrid=True, gridcolor='lightgray'))
fig.show()

## 4. Calibration
### 4.1 Error by timestamp

In [77]:
# Align dataframes based on 'timestamp', 'Tech'
gen_res['DispatchGen_MW_xm'] = gen_res['DispatchGen_MW']
gen_disp['DispatchGen_MW_sw'] = gen_disp['DispatchGen_MW']
gen_compare = pd.merge(gen_res, gen_disp, on=['timestamp', 'Tech'])
# Calculate error percentage
gen_compare['percentage_error (%)'] = 100*((gen_compare['DispatchGen_MW_sw'] / gen_compare['DispatchGen_MW_xm']) - 1)
# Remove rows where 'type' contains '_0h' and 'tech' is 'solar'
hours = r'_19h|_20h|_21h|_22h|_23h|_0h|_1h|_2h|_3h|_4h|_5h|_6h|_7h'
condition = ((gen_compare['Tech'] == 'Solar') & (gen_compare['timestamp'].str.contains(hours)))
gen_compare.loc[condition, ['percentage_error (%)']] = np.nan
#print(gen_compare.head(5))
import plotly.express as px
fig = px.line(
    gen_compare,
    x='timestamp', y='percentage_error (%)', 
    color='Tech')
fig.update_layout(plot_bgcolor='white',xaxis=dict(gridcolor='lightgray'),yaxis=dict(gridcolor='lightgray'))
fig.show()

### Error by Quartil

In [78]:
# Align dataframes based on 'timestamp', 'Tech'
gen_compare_Q = pd.merge(gen_res, gen_disp, on=['timestamp', 'Tech'])
gen_compare_Q['Quarter'] = gen_compare_Q['timestamp'].apply(lambda x: x[:7])

gen_compare_Q = gen_compare_Q.groupby(['Quarter','Tech']).agg({
    'DispatchGen_MW_xm': 'mean', 'DispatchGen_MW_sw' : 'mean'
}).reset_index()

# Calculate error percentage
gen_compare_Q['percentage_error (%)'] = 100 * ((gen_compare_Q['DispatchGen_MW_sw'] / gen_compare_Q['DispatchGen_MW_xm']) -1)
#print(gen_compare_Q.head(5))
import plotly.express as px
fig = px.line(
    gen_compare_Q, 
    x='Quarter', y='percentage_error (%)', 
    color='Tech')
fig.update_layout(plot_bgcolor='white',xaxis=dict(gridcolor='lightgray'),yaxis=dict(gridcolor='lightgray'))
fig.show()

In [79]:
# Alinear los dataframes usando merge
gen_compare_year = pd.merge(gen_res, gen_disp, on=['timestamp', 'Tech'])

gen_compare_year = gen_compare_Q.groupby(['Tech']).agg({
    'DispatchGen_MW_xm': 'mean', 'DispatchGen_MW_sw' : 'mean'
}).reset_index()

# Calcular el porcentaje
gen_compare_year['percentage'] = (gen_compare_year['DispatchGen_MW_sw'] / gen_compare_year['DispatchGen_MW_xm']) - 1
print('Error by Tech (Year)')
gen_compare_year

Error by Tech (Year)


,Tech,DispatchGen_MW_xm,DispatchGen_MW_sw,percentage
0,Eolica,18.755296,24.543755,0.308631
1,Hidro,6613.706746,6863.502624,0.037769
2,Menores,89.051693,131.747254,0.479447
3,Solar,122.227058,159.587176,0.305662
4,Thermal,1836.659252,1644.438106,-0.104658


In [80]:
# Promedio ponderado de GeneReal y percentage
weighted_avg_percentage = \
    (gen_compare_year['DispatchGen_MW_xm'] * gen_compare_year['percentage']).sum() / gen_compare_year['DispatchGen_MW_xm'].sum()
print(f"Average error percentage: {weighted_avg_percentage*100:.6f}%")

Average error percentage: 1.652215%


In [81]:
import pandas as pd
import plotly.express as px

model_outputs_path = '../../model/outputs/'

# Cargar el archivo CSV
emissions = pd.read_csv(model_outputs_path + 'emissions.csv')
# Filtrar las columnas necesarias
emissions = emissions[['PERIOD', 'AnnualEmissions_tCO2_per_yr']]
# Convertir las emisiones a millones
emissions['AnnualEmissions_tCO2_per_yr'] = emissions['AnnualEmissions_tCO2_per_yr'] / 1000000
# Crear el gráfico de barras interactivo
fig = px.bar(
    emissions, x='PERIOD', y='AnnualEmissions_tCO2_per_yr',
    labels={'AnnualEmissions_tCO2_per_yr': 'Annual Emissions (MtCO2)', 'PERIOD': 'Year'},
    text='AnnualEmissions_tCO2_per_yr')
# Mejorar la visualización
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis=dict(tickvals=emissions['PERIOD'].to_list()))
fig.show()

In [82]:
import pandas as pd
esc_ems = pd.read_csv('../../data/XM-API/Plan/emissions/esc0.csv')
fig = px.bar(
    esc_ems, x='PERIOD', y='AnnualEmissions_tCO2_per_yr',
    labels={'AnnualEmissions_tCO2_per_yr': 'Annual Emissions (MtCO2)', 'PERIOD': 'Year'},
    text='AnnualEmissions_tCO2_per_yr')
# Mejorar la visualización
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis=dict(tickvals=emissions['PERIOD'].to_list()))
fig.show()

In [83]:
import pandas as pd
model_outputs_path = '../../model/outputs/'
model_inputs_path = '../../model/inputs/'
cap_inst = pd.read_csv(model_outputs_path+'BuildGen.csv')

gen_info = pd.read_csv(model_inputs_path+'gen_info.csv')
#costs = gen_build_costs.merge(gen_info,on='GENERATION_PROJECT')
cap_inst = pd.merge(cap_inst, gen_info, left_on='GEN_BLD_YRS_1',
                    right_on='GENERATION_PROJECT', how='inner')
## Remove disabled projects
cap_inst = cap_inst[cap_inst['gen_max_age'] > 1]

cap_inst = cap_inst[['GENERATION_PROJECT','gen_tech','GEN_BLD_YRS_2','BuildGen']]
cap_inst_2023 = cap_inst[cap_inst['GEN_BLD_YRS_2'] <= 2023]
cap_inst_2023 = cap_inst_2023.groupby(['gen_tech']).agg({
    'BuildGen' : 'sum'}).reset_index()

cap_inst = cap_inst[cap_inst['GEN_BLD_YRS_2'] <= 2037]
cap_inst = cap_inst.groupby(['gen_tech']).agg({
    'BuildGen' : 'sum'}).reset_index()

# Create individual figures
fig1 = px.pie(cap_inst_2023, names='gen_tech', values='BuildGen', title='Generation Reported (XM)')
fig2 = px.pie(cap_inst, names='gen_tech', values='BuildGen', title='Generation Predicted (Switch)')

# Create subplots with the 'type' argument set to 'domain' to accommodate pie charts
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], 
                    subplot_titles=("2023", "2037"))

# Add figures to the subplots
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Ensure legend items show only once
names = set()
for trace in fig.data:
    if (trace.name in names):
        trace.showlegend = False
    else:
        names.add(trace.name)

# Show the figure
fig.show()

In [84]:
import pandas as pd
esc_cap_2023 = pd.read_csv('../../data/XM-API/Plan/installed_capacity/esc0/2023.csv')
esc_cap_2037 = pd.read_csv('../../data/XM-API/Plan/installed_capacity/esc0/2037.csv')

# Create individual figures
fig1 = px.pie(esc_cap_2023, names='gen_tech', values='BuildGen', title='Generation Reported (XM)')
fig2 = px.pie(esc_cap_2037, names='gen_tech', values='BuildGen', title='Generation Predicted (Switch)')

# Create subplots with the 'type' argument set to 'domain' to accommodate pie charts
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], 
                    subplot_titles=("2023", "2037"))

# Add figures to the subplots
for trace in fig1.data: fig.add_trace(trace, row=1, col=1)
for trace in fig2.data: fig.add_trace(trace, row=1, col=2)

# Ensure legend items show only once
names = set()
for trace in fig.data:
    if (trace.name in names): trace.showlegend = False
    else: names.add(trace.name)

# Show the figure
fig.show()